In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from internetarchive import search_items, get_item, Search

In [4]:
import requests
from bs4 import BeautifulSoup

In [8]:
from ner_pipeline.scrape_for_training import prepare_data, do_search

In [9]:
# Do search
il_od: str = "iliad OR odyssey AND mediatype:texts"  # 771,646 with full_text_search, 6240 without
search_res: Search = do_search(keyword_string=il_od)

Search string: iliad OR odyssey AND mediatype:texts
Results: 538896


In [10]:
# pattern = r'Iliad\s\d{1,2}\.\d{1,4}|Il\.*\s\d{1,2}\.\d{1,4}|Iliad\s.[ivxlcdm]*\.\s*\d{1,4}| \
# Il\.*\s.[ivxlcdm]*\.\s*\d{1,4}|book\s*.[ivxlcdm]\.\sline\s*\d{1,4}|Odyssey\s\d{1,2}\.\d{1,4}| \
# Od\.*\s\d{1,2}\.\d{1,4}|Odyssey\s.[ivxlcdm]*\.\s*\d{1,4}|Od\.*\s.[ivxlcdm]*\.\s*\d{1,4}'

In [11]:
# dataset = prepare_data(search_res, pattern)

In [12]:
# print(dataset[:10])

In [13]:
i = 0
for item in search_res:
    if i <= 10:
        book_url = "https://archive.org/details/" + item["fields"]["identifier"][0]
        #txt_url = "https://archive.org/stream/" + item["identifier"] + "/" + item["identifier"] + "_djvu.txt"
        print(book_url)
    else:
        break
    i+=1

https://archive.org/details/dictionaryofclas0000zimm_i4j1
https://archive.org/details/introductionsto00cole
https://archive.org/details/jstor-261886
https://archive.org/details/homeranintroduc04jebbgoog
https://archive.org/details/cu31924059064653
https://archive.org/details/homeranintroduc00jebbgoog
https://archive.org/details/homerodyssey0000said
https://archive.org/details/problemhomericp01geddgoog
https://archive.org/details/artandhumanityi00lawtgoog
https://archive.org/details/dli.granth.40696
https://archive.org/details/problemhomericp00unkngoog


In [16]:
# Get the url to .txt file
book_web_page = requests.get(book_url)

In [17]:
book_web_page_soup = BeautifulSoup(book_web_page.text, 'html.parser')

txt_url = ""
for link in book_web_page_soup.find_all('a', 'stealth'):
    txt_url_candidate = link.get('href')
    print(txt_url_candidate)
    if txt_url_candidate.endswith('_djvu.txt'):
        txt_url = txt_url_candidate
        
if txt_url:
    txt_url = 'https://archive.org' + txt_url
    print(txt_url)
else:
    print("No text file available.")

/write-review.php?identifier=problemhomericp00unkngoog
/download/problemhomericp00unkngoog/problemhomericp00unkngoog_abbyy.gz
/download/problemhomericp00unkngoog/problemhomericp00unkngoog_daisy.zip
/download/problemhomericp00unkngoog/problemhomericp00unkngoog.epub
/stream/problemhomericp00unkngoog/problemhomericp00unkngoog_djvu.txt
/download/problemhomericp00unkngoog/__ia_thumb.jpg
/download/problemhomericp00unkngoog/problemhomericp00unkngoog.mobi
/download/problemhomericp00unkngoog/problemhomericp00unkngoog.pdf
/download/problemhomericp00unkngoog/problemhomericp00unkngoog_tif.zip
/download/problemhomericp00unkngoog/problemhomericp00unkngoog_archive.torrent
/about/terms.php
https://archive.org/stream/problemhomericp00unkngoog/problemhomericp00unkngoog_djvu.txt


In [27]:
txt_web_page = requests.get(txt_url)
#txt_web_page.text

In [15]:
txt_web_page_soup = BeautifulSoup(txt_web_page.text, 'html.parser')

book_content = txt_web_page_soup.find_all('pre')[0]

print(type(book_content.text))


<class 'str'>


In [16]:
lines = book_content.text.split('\n')
lines = [line for line in lines if line != '']
print(len(lines))

13366


In [22]:
pattern = r'Iliad\s\d{1,2}\.\d{1,4}|Il\.*\s\d{1,2}\.\d{1,4}|Iliad\s.[ivxlcdm]*\.\s*\d{1,4}| \
Il\.*\s.[ivxlcdm]*\.\s*\d{1,4}|book\s*.[ivxlcdm]\.\sline\s*\d{1,4}|Odyssey\s\d{1,2}\.\d{1,4}| \
Od\.*\s\d{1,2}\.\d{1,4}|Odyssey\s.[ivxlcdm]*\.\s*\d{1,4}|Od\.*\s.[ivxlcdm]*\.\s*\d{1,4}'
dataset=(prepare_data(lines, pattern))
dataset

[{'content': '.^/.. ', 'annotations': []},
 {'content': '.-f ', 'annotations': []},
 {'content': '.-jt. ', 'annotations': []},
 {'content': 'BOUGHT WITH THE INCOME ', 'annotations': []},
 {'content': 'FROM THE ', 'annotations': []},
 {'content': 'SAGE ENDOWMENT FUND ', 'annotations': []},
 {'content': 'THE GIFT OF ', 'annotations': []},
 {'content': 'munvQ W. Sage ', 'annotations': []},
 {'content': '1891 ', 'annotations': []},
 {'content': "^ ^^■^■7^g^' ;^ : -U/T^/.c':.. ", 'annotations': []},
 {'content': '^ 6896-1 ', 'annotations': []},
 {'content': "OUNUBRARY-CIRCULAT'C.. ", 'annotations': []},
 {'content': 'IDATE DUE ', 'annotations': []},
 {'content': '-PP^-r-^fW-H, ', 'annotations': []},
 {'content': '.«)^ ', 'annotations': []},
 {'content': "-' ", 'annotations': []},
 {'content': '* o ', 'annotations': []},
 {'content': '^^6 ', 'annotations': []},
 {'content': '- ', 'annotations': []},
 {'content': '\'"■^^^ ', 'annotations': []},
 {'content': '36 AP si-f» ', 'annotations': []},

In [25]:
count = 0
for i in dataset:
    if i['annotations'] != []:
        print(i)
        count += 1
print(count)

{'content': '^/zW minstrel (Od. i. 351 ; viii. 64). On the other hand, it is quite possible ', 'annotations': [{'start': 15, 'end': 25, 'label': 'CLEntity'}]}
{'content': 'signal for the end (Od. viii. 75-82). The terms of the ', 'annotations': [{'start': 20, 'end': 32, 'label': 'CLEntity'}]}
{'content': 'vii. i2i,xii. 213 ; Od. xiv. 509, iv. 348. Q>. II. vi. 333, x. 445 ; Hes.Op.260). ', 'annotations': [{'start': 20, 'end': 32, 'label': 'CLEntity'}]}
{'content': '\' Cp. xii. 237-243 ; Od. xix. 560, 561. * "Companion," p. 67. ', 'annotations': [{'start': 21, 'end': 33, 'label': 'CLEntity'}]}
{'content': 'wise (Od. vii. 73-74). Achilles almost admits as much in ', 'annotations': [{'start': 6, 'end': 17, 'label': 'CLEntity'}]}
{'content': 'called a /xeyapov (Od. xvi. 165). The Myrmidons could ', 'annotations': [{'start': 19, 'end': 31, 'label': 'CLEntity'}]}
{'content': 'son put the weapons away (Od. xvi. 294, repeated xix. ', 'annotations': [{'start': 26, 'end': 38, 'label': 'CLEntity'}